In [49]:
import os
import pandas as pd
from pan22_verif_evaluator import evaluate_all
from cngdist import correct_scores
import numpy as np

In [75]:
distortion_dirs = [
    "DV-MA-k-300",
    "DV-MA-k-3000",
    "DV-MA-k-20000",
    "undistorted"
]

params = pd.read_csv('training_metrics.csv')

results = {"Dataset": [],
           "k": [],
           "p1": [],
           "p2": [],
           "auc": [],
           "c@1": [],
           "f_05_u": [],
           "F1": [],
           "brier": [],
           "overall": []}
num_splits = 5
for distortion_dir in distortion_dirs:
    for k in range(num_splits):
        path_to_val_truths = f"./data/validate/{distortion_dir}/k_{k}/truth.jsonl"
        path_to_val_preds = f"./out/{distortion_dir}/k_{k}/answers.jsonl"

        row = params[(params['Dataset'] == distortion_dir) & (params['k'] == f'k_{k}')]
        p1, p2 = row['p1'][row.index[0]], row['p2'][row.index[0]]

        df_a = pd.read_json(path_to_val_preds, lines=True)['value']
        similarities = df_a.tolist()
        corrected_scores = np.array(
            list(correct_scores(similarities, p1=p1, p2=p2)))

        labels = pd.read_json(path_to_val_truths, lines=True)['same'].to_numpy()
        score = evaluate_all(pred_y=corrected_scores, true_y=labels)
        results["auc"].append(score["auc"])
        results["c@1"].append(score["c@1"])
        results["f_05_u"].append(score["f_05_u"])
        results["F1"].append(score["F1"])
        results["brier"].append(score["brier"])
        results["overall"].append(score["overall"])
        results['Dataset'].append(distortion_dir)
        results['k'].append(f'k_{k}')
        results['p1'].append(p1)
        results['p2'].append(p2)
results = pd.DataFrame(results)
results.to_csv('validation_metrics.csv', index=False)

/Users/zacbolton/dev/BSc/FP/historical_av_with_SBERT/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/zacbolton/dev/BSc/FP/historical_av_with_SBERT/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/zacbolton/dev/BSc/FP/historical_av_with_SBERT/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control th